# Importing the libraries and loading the dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [58]:
df = pd.read_csv('2024_Accidentalidad.csv', sep=';')

In [6]:
df.sample(5)

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga
27052,2024S022309,17/07/2024,15:15:00,"AUTOV. A-2, +00500E",+00500E,15,CIUDAD LINEAL,Colisión lateral,Despejado,Furgoneta,Pasajero,De 25 a 29 años,Mujer,14.0,Sin asistencia sanitaria,444520.0,4477574.0,N,NaN
11555,2024S010959,27/03/2024,11:19:00,CALL. ANDRES OBISPO / CALL. AGUSTIN CALVO,20,16,HORTALEZA,Colisión fronto-lateral,Despejado,Turismo,Pasajero,De 60 a 64 años,Mujer,5.0,Asistencia sanitaria ambulatoria con posterior...,445055.0,4479489.0,N,NaN
7594,2024S007280,27/02/2024,21:50:00,"CALL. DOCTOR SEVERO OCHOA, 1",1,9,MONCLOA-ARAVACA,Choque contra obstáculo fijo,NaN,Turismo,Conductor,Desconocido,Desconocido,NaN,NaN,438523.0,4477072.0,N,NaN
16994,2024S015709,07/05/2024,17:20:00,CALL. REAL DE ARGANDA / AVDA. ENSANCHE DE VALL...,64,18,VILLA DE VALLECAS,Colisión fronto-lateral,Despejado,Turismo,Conductor,De 45 a 49 años,Mujer,14.0,Sin asistencia sanitaria,447887.0,4470047.0,N,NaN
29896,2024S024098,13/08/2024,15:15:00,"CALL. PALOMARES, 6",6,17,VILLAVERDE,Alcance,Despejado,Furgoneta,Conductor,De 35 a 39 años,Hombre,NaN,NaN,439829.0,4466593.0,N,NaN


In [12]:
df.shape

(40165, 19)

# Tarea 1
*Realiza un análisis descriptivo del dataset. Analiza la distribución de los datos
por cada una de las columnas, realiza los pasos de pre-procesamiento necesarios,
justificando adecuadamente las acciones tomadas. Se deberá hacer uso de gráficas para
entender los datos y las decisiones adoptadas.*

## 1. Preprocess

### 1.1. NaN

In [13]:
def null_percentage(df, percentage):
    null_ratio = {}
    for col in df.columns:
        ratio = df[col].isna().sum() / len(df) * 100
        if ratio > percentage:
            null_ratio[col] = ratio
           
    return null_ratio

In [15]:
null_percentage(df, 0)

{'tipo_accidente': 0.007469189592929167,
 'estado_meteorológico': 11.597161707954687,
 'tipo_vehiculo': 0.799203286443421,
 'cod_lesividad': 44.38939375077804,
 'lesividad': 44.38939375077804,
 'coordenada_x_utm': 0.024897298643097227,
 'coordenada_y_utm': 0.024897298643097227,
 'positiva_alcohol': 0.34358272127474165,
 'positiva_droga': 99.57674592306735}

According to Adrian: 
- everything that is lower than 10 -> we drop the rows
- estado meteorológico -> try to inpute the data
- drogas -> drop the column
- lesividad -> create a new category to encode the NaNs

### 1.2. Redundant data

There are two pairs of columns that contain exactly the same information: *distrito* and *cod_distrito* and *lesividad* and *cod_lesividad*.

We decided to drop the columns containg names, as the others are already "encoded". However, first we will create dictionaries that will facilitate the interpretation of the data. We will also change the codes to numeric values, so that we can use them in later analysis.

#### 1.2.1. Distrito

At first, we have to see if the codes align with the names.

In [17]:
df['distrito'].value_counts()

distrito
PUENTE DE VALLECAS     3253
CHAMARTÍN              2864
CARABANCHEL            2862
SALAMANCA              2636
CIUDAD LINEAL          2448
CENTRO                 2254
ARGANZUELA             2175
FUENCARRAL-EL PARDO    2032
RETIRO                 2013
MONCLOA-ARAVACA        1990
LATINA                 1934
TETUÁN                 1894
SAN BLAS-CANILLEJAS    1884
CHAMBERÍ               1788
USERA                  1718
HORTALEZA              1473
VILLAVERDE             1426
MORATALAZ              1093
VILLA DE VALLECAS      1046
BARAJAS                 711
VICÁLVARO               671
Name: count, dtype: int64

In [18]:
df['cod_distrito'].value_counts()

cod_distrito
13    3253
5     2864
11    2862
4     2636
15    2448
1     2254
2     2175
8     2032
3     2013
9     1990
10    1934
6     1894
20    1884
7     1788
12    1718
16    1473
17    1426
14    1093
18    1046
21     711
19     671
Name: count, dtype: int64

We can see that the codes in the *cod_distrito* column are assigned taking into account the city zone, so the distance is preserved. 

Now, we create the dictionary contaning district names and codes.

In [ ]:
district_keys = df['cod_distrito'].unique()    
district_names = df['distrito'].unique()

district_dict = dict(zip(district_keys, district_names))


[16  1  7  6 10  9  3 13 14 11  4  5 19  2  8 12 18 21 17 20 15]
['HORTALEZA' 'CENTRO' 'CHAMBERÍ' 'TETUÁN' 'LATINA' 'MONCLOA-ARAVACA'
 'RETIRO' 'PUENTE DE VALLECAS' 'MORATALAZ' 'CARABANCHEL' 'SALAMANCA'
 'CHAMARTÍN' 'VICÁLVARO' 'ARGANZUELA' 'FUENCARRAL-EL PARDO' 'USERA'
 'VILLA DE VALLECAS' 'BARAJAS' 'VILLAVERDE' 'SAN BLAS-CANILLEJAS'
 'CIUDAD LINEAL']


In [37]:
district_dict

{16: 'HORTALEZA',
 1: 'CENTRO',
 7: 'CHAMBERÍ',
 6: 'TETUÁN',
 10: 'LATINA',
 9: 'MONCLOA-ARAVACA',
 3: 'RETIRO',
 13: 'PUENTE DE VALLECAS',
 14: 'MORATALAZ',
 11: 'CARABANCHEL',
 4: 'SALAMANCA',
 5: 'CHAMARTÍN',
 19: 'VICÁLVARO',
 2: 'ARGANZUELA',
 8: 'FUENCARRAL-EL PARDO',
 12: 'USERA',
 18: 'VILLA DE VALLECAS',
 21: 'BARAJAS',
 17: 'VILLAVERDE',
 20: 'SAN BLAS-CANILLEJAS',
 15: 'CIUDAD LINEAL'}

Now, we drop the *distrito* column.

In [46]:
df.drop(['distrito'], axis=1, inplace=True)

#### 1.2.2. Lesividad

We check, if the values align.

In [21]:
df['cod_lesividad'].value_counts()

cod_lesividad
14.0    13109
7.0      4720
2.0      1494
1.0      1111
6.0       950
3.0       475
5.0       453
4.0        24
Name: count, dtype: int64

In [22]:
df['lesividad'].value_counts()

lesividad
Sin asistencia sanitaria                                     13109
Asistencia sanitaria sólo en el lugar del accidente           4720
Ingreso inferior o igual a 24 horas                           1494
Atención en urgencias sin posterior ingreso                   1111
Asistencia sanitaria inmediata en centro de salud o mutua      950
Ingreso superior a 24 horas                                    475
Asistencia sanitaria ambulatoria con posterioridad             453
Fallecido 24 horas                                              24
Name: count, dtype: int64

In [60]:
df['lesividad'].unique(), df['cod_lesividad'].unique()

(array(['Ingreso inferior o igual a 24 horas', 'Sin asistencia sanitaria',
        'Asistencia sanitaria sólo en el lugar del accidente', nan,
        'Atención en urgencias sin posterior ingreso',
        'Ingreso superior a 24 horas',
        'Asistencia sanitaria inmediata en centro de salud o mutua',
        'Asistencia sanitaria ambulatoria con posterioridad',
        'Fallecido 24 horas'], dtype=object),
 array([ 2., 14.,  7., nan,  1.,  3.,  6.,  5.,  4.]))

As we know, those columns containg more than 40% of NaN values.

We could treat them as another category and encode them as well, naming it *'sin información'*. Inputing 40% of data uing other 60% does not really make sense. Nor does dropping almost half of the rows. So we decided to use encoding, as it is the most reasonbale option.

!!!!! Those columns contain null values and, as we are going to hadle them later, we will now create a dictionary that won't contain NaNs.

In [ ]:
# we are going to handle nans later, so we don't include them in the dictionary
lesividad_keys = df['cod_lesividad'].dropna().unique().tolist()

lesividad_names = df['lesividad'].dropna().unique().tolist()

lesividad_dict = dict(zip(lesividad_keys, lesividad_names))
print(lesividad_dict)


[2.0, 14.0, 7.0, 1.0, 3.0, 6.0, 5.0, 4.0]
['Ingreso inferior o igual a 24 horas', 'Sin asistencia sanitaria', 'Asistencia sanitaria sólo en el lugar del accidente', 'Atención en urgencias sin posterior ingreso', 'Ingreso superior a 24 horas', 'Asistencia sanitaria inmediata en centro de salud o mutua', 'Asistencia sanitaria ambulatoria con posterioridad', 'Fallecido 24 horas']
{2.0: 'Ingreso inferior o igual a 24 horas', 14.0: 'Sin asistencia sanitaria', 7.0: 'Asistencia sanitaria sólo en el lugar del accidente', 1.0: 'Atención en urgencias sin posterior ingreso', 3.0: 'Ingreso superior a 24 horas', 6.0: 'Asistencia sanitaria inmediata en centro de salud o mutua', 5.0: 'Asistencia sanitaria ambulatoria con posterioridad', 4.0: 'Fallecido 24 horas'}


Now, we can drop the *lesividad* column.

In [55]:
df.drop(['lesividad'], axis=1, inplace=True)

In [24]:
df.sample(5)

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga
36501,2024S028091,07/10/2024,13:10:00,CALL. BUSTAMANTE / CALL. VILLA DEL PRADO,35,2,Colisión lateral,Despejado,Furgoneta,Pasajero,De 50 a 54 años,Hombre,NaN,441737.0,4472434.0,N,NaN
32178,2024S025554,07/09/2024,10:00:00,CALL. CERRO MINGUETE / CALL. RISCOS DE POLANCO,96,8,Colisión fronto-lateral,Despejado,Turismo,Conductor,De 50 a 54 años,Hombre,14.0,438752.0,4482044.0,N,NaN
24229,2024S020697,26/06/2024,15:02:00,"AVDA. LOGROÑO, 305",305,21,Colisión lateral,Nublado,Turismo,Pasajero,De 25 a 29 años,Mujer,14.0,450684.0,4480075.0,N,NaN
1236,2024S000920,11/01/2024,16:10:00,CALL. ARTURO SORIA / CALL. CALERUEGA,316,15,Colisión fronto-lateral,Despejado,Turismo,Conductor,De 60 a 64 años,Hombre,14.0,443314.0,4480944.0,N,NaN
6170,2024S005808,16/02/2024,15:40:00,CALL. ARROYO BUENO / AVDA. REAL DE PINTO,5,17,Alcance,Despejado,Turismo,Conductor,De 60 a 64 años,Hombre,NaN,440037.0,4466761.0,N,NaN


In [25]:
df.shape

(40165, 17)

We want to check if every ID (*num_expediente* column) corresponds to only one row.

In [27]:
len(df['num_expediente'].unique())

16935

It is obviously false, as there are usually one there more vehicule/person involved in the accident.